# Chapter 7 : Basic Statistics in R
This chapter mainly introduce some methods describing basic statistic
 - descriptive statistics
 - Frequency and contingency tables
 - Correlations and covariances
 - t-tests
 - Nonparamettic statistics

## descriptive statistics
### sample data

In [1]:
vars = c("mpg", "hp", "wt")
head(mtcars[vars])
summary(mtcars[vars])   # basic information of data

,mpg,hp,wt
Mazda RX4,21.0,110,2.620
Mazda RX4 Wag,21.0,110,2.875
Datsun 710,22.8,93,2.320
Hornet 4 Drive,21.4,110,3.215
Hornet Sportabout,18.7,175,3.440
Valiant,18.1,105,3.460


      mpg              hp              wt       
 Min.   :10.40   Min.   : 52.0   Min.   :1.513  
 1st Qu.:15.43   1st Qu.: 96.5   1st Qu.:2.581  
 Median :19.20   Median :123.0   Median :3.325  
 Mean   :20.09   Mean   :146.7   Mean   :3.217  
 3rd Qu.:22.80   3rd Qu.:180.0   3rd Qu.:3.610  
 Max.   :33.90   Max.   :335.0   Max.   :5.424  

### using sapply

In [2]:
## skew, kurt
mystats = function(x, na.omit=FALSE){
  if (na.omit)
    x = x[!is.na(x)]
  m = mean(x)
  n = length(x)
  s = sd(x)
  skew = sum((x-m)^3/s^3)/n
  kurt = sum((x-m)^4/s^4)/n - 3
  return(c(n=n, mean=m, stdev=s, skew=skew, kurosis=kurt))
}
sapply(mtcars[vars], mystats)

,mpg,hp,wt
n,32.000000,32.0000000,32.00000000
mean,20.090625,146.6875000,3.21725000
stdev,6.026948,68.5628685,0.97845744
skew,0.610655,0.7260237,0.42314646
kurosis,-0.372766,-0.1355511,-0.02271075


### using package pastecs

In [3]:
library(pastecs)
stat.desc(mtcars[vars])

,mpg,hp,wt
nbr.val,32.0000000,32.0000000,32.0000000
nbr.null,0.0000000,0.0000000,0.0000000
nbr.na,0.0000000,0.0000000,0.0000000
min,10.4000000,52.0000000,1.5130000
max,33.9000000,335.0000000,5.4240000
range,23.5000000,283.0000000,3.9110000
sum,642.9000000,4694.0000000,102.9520000
median,19.2000000,123.0000000,3.3250000
mean,20.0906250,146.6875000,3.2172500
SE.mean,1.0654240,12.1203173,0.1729685


### using package psych

In [4]:
library(psych)
describe(mtcars[vars])

,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
mpg,1,32,20.09062,6.0269481,19.200,19.696154,5.4114900,10.400,33.900,23.500,0.6106550,-0.37276603,1.0654240
hp,2,32,146.68750,68.5628685,123.000,141.192308,77.0952000,52.000,335.000,283.000,0.7260237,-0.13555112,12.1203173
wt,3,32,3.21725,0.9784574,3.325,3.152692,0.7672455,1.513,5.424,3.911,0.4231465,-0.02271075,0.1729685


## Descriptive statistics by group

### using aggregate

In [6]:
aggregate(mtcars[vars], by=list(am=mtcars$am), mean)
aggregate(mtcars[vars], by=list(am=mtcars$am), sd)

am,mpg,hp,wt
0,17.14737,160.2632,3.768895
1,24.39231,126.8462,2.411000


am,mpg,hp,wt
0,3.833966,53.90820,0.7774001
1,6.166504,84.06232,0.6169816


### using by 
Attention: Some error in by() function

In [8]:
dstats = function(x)(c(mean=mean(x), sd=sd(x)))
by(mtcars[vars], mtcars$am, sd)

ERROR: Error in is.data.frame(x): (list) object cannot be coerced to type 'double'


### using package psych

In [10]:
### psych
describe.by(mtcars[vars], mtcars$am)

Warning message:
“describe.by is deprecated.  Please use the describeBy function”


 Descriptive statistics by group 
group: 0
    vars  n   mean    sd median trimmed   mad   min    max  range  skew
mpg    1 19  17.15  3.83  17.30   17.12  3.11 10.40  24.40  14.00  0.01
hp     2 19 160.26 53.91 175.00  161.06 77.10 62.00 245.00 183.00 -0.01
wt     3 19   3.77  0.78   3.52    3.75  0.45  2.46   5.42   2.96  0.98
    kurtosis    se
mpg    -0.80  0.88
hp     -1.21 12.37
wt      0.14  0.18
------------------------------------------------------------ 
group: 1
    vars  n   mean    sd median trimmed   mad   min    max  range skew kurtosis
mpg    1 13  24.39  6.17  22.80   24.38  6.67 15.00  33.90  18.90 0.05    -1.46
hp     2 13 126.85 84.06 109.00  114.73 63.75 52.00 335.00 283.00 1.36     0.56
wt     3 13   2.41  0.62   2.32    2.39  0.68  1.51   3.57   2.06 0.21    -1.17
       se
mpg  1.71
hp  23.31
wt   0.17

### using reshape and cast function

In [12]:
library(reshape)
dstats = function(x)(c(n=length(x),mean=mean(x), sd=sd(x)))
dfm = melt(mtcars, measure.vars=c("mpg", "hp", "wt"), id.vars = c("am", "cyl"))
cast(dfm, am+cyl+variable ~ ., dstats)

am,cyl,variable,n,mean,sd
0,4,mpg,3,22.900000,1.4525839
0,4,hp,3,84.666667,19.6553640
0,4,wt,3,2.935000,0.4075230
0,6,mpg,4,19.125000,1.6317169
0,6,hp,4,115.250000,9.1787799
0,6,wt,4,3.388750,0.1162164
0,8,mpg,12,15.050000,2.7743959
0,8,hp,12,194.166667,33.3598379
0,8,wt,12,4.104083,0.7683069
1,4,mpg,8,28.075000,4.4838599


## Visualizing result
### using Arthritis data

In [14]:
library(vcd)
head(Arthritis)

ID,Treatment,Sex,Age,Improved
57,Treated,Male,27,Some
46,Treated,Male,29,None
77,Treated,Male,30,None
17,Treated,Male,32,Marked
36,Treated,Male,46,Marked
23,Treated,Male,58,Marked


### Frequency table

In [18]:
mytable = with(Arthritis, table(Improved))
mytable
prop.table(mytable)

Improved
  None   Some Marked 
    42     14     28 

Improved
     None      Some    Marked 
0.5000000 0.1666667 0.3333333 

### Margin table
Using table and xlabs

In [20]:
mytable = xtabs(~ Treatment+Improved, data=Arthritis)
margin.table(mytable, 1)
prop.table(mytable, 1)
prop.table(mytable)   # element / all

Treatment
Placebo Treated 
     43      41 

         Improved
Treatment      None      Some    Marked
  Placebo 0.6744186 0.1627907 0.1627907
  Treated 0.3170732 0.1707317 0.5121951

         Improved
Treatment       None       Some     Marked
  Placebo 0.34523810 0.08333333 0.08333333
  Treated 0.15476190 0.08333333 0.25000000

### Add margins in table

In [22]:
addmargins(mytable)
addmargins(prop.table(mytable, 1), 2)
addmargins(prop.table(mytable, 2), 1)

,None,Some,Marked,Sum
Placebo,29,7,7,43
Treated,13,7,21,41
Sum,42,14,28,84


,None,Some,Marked,Sum
Placebo,0.6744186,0.1627907,0.1627907,1
Treated,0.3170732,0.1707317,0.5121951,1


,None,Some,Marked
Placebo,0.6904762,0.5,0.25
Treated,0.3095238,0.5,0.75
Sum,1.0000000,1.0,1.00


## Test of dependency
### Using package vcd

In [24]:
library(vcd)

### Chisq test

In [26]:
mytable = xtabs(~Treatment+Improved, data = Arthritis)
chisq.test(mytable)
mytable = xtabs(~Improved+Sex, data = Arthritis)
chisq.test(mytable)


	Pearson's Chi-squared test

data:  mytable
X-squared = 13.055, df = 2, p-value = 0.001463


Warning message in chisq.test(mytable):
“Chi-squared approximation may be incorrect”


	Pearson's Chi-squared test

data:  mytable
X-squared = 4.8407, df = 2, p-value = 0.08889


### Fisher test

In [28]:
mytable = xtabs(~Treatment+Improved, data = Arthritis)
fisher.test(mytable)


	Fisher's Exact Test for Count Data

data:  mytable
p-value = 0.001393
alternative hypothesis: two.sided


### Cochran-Mantel-Haenszel test

In [30]:
mytable = xtabs(~Treatment+Improved+Sex, data = Arthritis)
mantelhaen.test(mytable)


	Cochran-Mantel-Haenszel test

data:  mytable
Cochran-Mantel-Haenszel M^2 = 14.632, df = 2, p-value = 0.0006647


### Test of association
Using Phi-Coefficient, Contingency Coeff and Cramer's V describe

In [32]:
mytable = xtabs(~Treatment+Improved, data = Arthritis)
assocstats(mytable)

                    X^2 df  P(> X^2)
Likelihood Ratio 13.530  2 0.0011536
Pearson          13.055  2 0.0014626

Phi-Coefficient   : NA 
Contingency Coeff.: 0.367 
Cramer's V        : 0.394 

### table to flat

In [34]:
table2flat = function(mytable) {
    df = as.data.frame(mytable)
    rows = dim(df)[1]
    cols = dim(df)[2]
    x = NULL
    for (i in 1:rows){
        for (j in 1: cols){
            row = df[i, c(1:(cols-1))]
            x = rbind(x, row)
        }
    }
    row.names(x) = c(1:dim(x)[1])
    return(x)
}

In [41]:
treatment = rep(c("Placebo", "Treated"), times=3)
improved = rep(c("None", "Some", "Marked"), each=2)
Freq = c(29, 13, 7, 17, 21)
mytable = as.data.frame(cbind(treatment, improved, Freq))
head(mytable)
mydata = table2flat(mytable)
head(mydata)

Warning message in cbind(treatment, improved, Freq):
“number of rows of result is not a multiple of vector length (arg 3)”

treatment,improved,Freq
Placebo,None,29
Treated,None,13
Placebo,Some,7
Treated,Some,17
Placebo,Marked,21
Treated,Marked,29


treatment,improved
Placebo,None
Placebo,None
Placebo,None
Treated,None
Treated,None
Treated,None


## Correlations
Some Correlation calculation in R, include Pearson, Spearman, Kendall, partial, polychoric, polyserrial

### Pearson, Spearman, Kendall
 - Pearson积差相关系数衡量了两个定量变量之间的线性相关程度
 - Spearman等级相关系数则衡量分级定序变量之间的相关程度
 - Kendall's Tau相关系数也是一种非参数的等级相关度量
 
 Using cor() to calculate coff. mentioned before and using cov() to calculate co-variance

In [44]:
states = state.x77[, 1:6]
head(states)

,Population,Income,Illiteracy,Life Exp,Murder,HS Grad
Alabama,3615,3624,2.1,69.05,15.1,41.3
Alaska,365,6315,1.5,69.31,11.3,66.7
Arizona,2212,4530,1.8,70.55,7.8,58.1
Arkansas,2110,3378,1.9,70.66,10.1,39.9
California,21198,5114,1.1,71.71,10.3,62.6
Colorado,2541,4884,0.7,72.06,6.8,63.9


### cov with each column

In [45]:
cov(states)

,Population,Income,Illiteracy,Life Exp,Murder,HS Grad
Population,19931683.7588,571229.7796,292.8679592,-407.8424612,5663.523714,-3551.509551
Income,571229.7796,377573.3061,-163.7020408,280.6631837,-521.894286,3076.768980
Illiteracy,292.8680,-163.7020,0.3715306,-0.4815122,1.581776,-3.235469
Life Exp,-407.8425,280.6632,-0.4815122,1.8020204,-3.869480,6.312685
Murder,5663.5237,-521.8943,1.5817755,-3.8694804,13.627465,-14.549616
HS Grad,-3551.5096,3076.7690,-3.2354694,6.3126849,-14.549616,65.237894


### cor with each column

In [46]:
cor(states)

,Population,Income,Illiteracy,Life Exp,Murder,HS Grad
Population,1.00000000,0.2082276,0.1076224,-0.06805195,0.3436428,-0.09848975
Income,0.20822756,1.0000000,-0.4370752,0.34025534,-0.2300776,0.61993232
Illiteracy,0.10762237,-0.4370752,1.0000000,-0.58847793,0.7029752,-0.65718861
Life Exp,-0.06805195,0.3402553,-0.5884779,1.00000000,-0.7808458,0.58221620
Murder,0.34364275,-0.2300776,0.7029752,-0.78084575,1.0000000,-0.48797102
HS Grad,-0.09848975,0.6199323,-0.6571886,0.58221620,-0.4879710,1.00000000


In [49]:
cor(states, method = 'kendall')

,Population,Income,Illiteracy,Life Exp,Murder,HS Grad
Population,1.00000000,0.08408163,0.2123063,-0.06865555,0.2364983,-0.2353905
Income,0.08408163,1.00000000,-0.1970811,0.21904389,-0.1448450,0.3579896
Illiteracy,0.21230629,-0.19708113,1.0000000,-0.42852098,0.5155359,-0.5047401
Life Exp,-0.06865555,0.21904389,-0.4285210,1.00000000,-0.5997547,0.3952537
Murder,0.23649826,-0.14484495,0.5155359,-0.59975465,1.0000000,-0.2884066
HS Grad,-0.23539045,0.35798964,-0.5047401,0.39525368,-0.2884066,1.0000000


### Partial correlations
 - using package ggm, functoin pcor(u, S),其中的u是一个数值向量，前两个数值表示要计算相关系数的变量下标，其余的数值为条件变量 (即要排除影响的变量)的下标
 - 偏相关性是指在控制了一个或多个定量变量时， 另外两个定量变量之间的相互关系

In [51]:
library(ggm)
pcor(c(1,5,2,3,6), cov(states))

[1] 0.3462724

### Testing Correlations for significance

#### Testing a correlation coefficient for significance

In [52]:
cor.test(states[,3], states[,5])


	Pearson's product-moment correlation

data:  states[, 3] and states[, 5]
t = 6.8479, df = 48, p-value = 1.258e-08
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.5279280 0.8207295
sample estimates:
      cor 
0.7029752 


#### Correlation matrix and tests of significance via corr.test()

In [53]:
library(psych)
corr.test(states, use="complete")

Call:corr.test(x = states, use = "complete")
Correlation matrix 
           Population Income Illiteracy Life Exp Murder HS Grad
Population       1.00   0.21       0.11    -0.07   0.34   -0.10
Income           0.21   1.00      -0.44     0.34  -0.23    0.62
Illiteracy       0.11  -0.44       1.00    -0.59   0.70   -0.66
Life Exp        -0.07   0.34      -0.59     1.00  -0.78    0.58
Murder           0.34  -0.23       0.70    -0.78   1.00   -0.49
HS Grad         -0.10   0.62      -0.66     0.58  -0.49    1.00
Sample Size 
[1] 50
Probability values (Entries above the diagonal are adjusted for multiple tests.) 
           Population Income Illiteracy Life Exp Murder HS Grad
Population       0.00   0.59       1.00      1.0   0.10       1
Income           0.15   0.00       0.01      0.1   0.54       0
Illiteracy       0.46   0.00       0.00      0.0   0.00       0
Life Exp         0.64   0.02       0.00      0.0   0.00       0
Murder           0.01   0.11       0.00      0.0   0.00       0
H

## T-tests

### Independent Sample

In [54]:
library(MASS)
t.test(Prob ~ So, data = UScrime)


	Welch Two Sample t-test

data:  Prob by So
t = -3.8954, df = 24.925, p-value = 0.0006506
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.03852569 -0.01187439
sample estimates:
mean in group 0 mean in group 1 
     0.03851265      0.06371269 


### Denpendent Sample

In [55]:
library(MASS)
sapply(UScrime[c("U1", "U2")], function(x)(c(mean=mean(x), sd=sd(x))))

,U1,U2
mean,95.46809,33.97872
sd,18.02878,8.44545


In [56]:
with(UScrime, t.test(U1, U2, paired = TRUE))


	Paired t-test

data:  U1 and U2
t = 32.407, df = 46, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 57.67003 65.30870
sample estimates:
mean of the differences 
               61.48936 


## Nonparametric tests of group differences
### Compare two group
 - Wilcoxon rank sum test (Mann-Whitney U test)

In [57]:
with(UScrime, by(Prob, So, median))

So: 0
[1] 0.038201
------------------------------------------------------------ 
So: 1
[1] 0.055552

In [58]:
wilcox.test(Prob ~ So, data=UScrime)


	Wilcoxon rank sum test

data:  Prob by So
W = 81, p-value = 8.488e-05
alternative hypothesis: true location shift is not equal to 0


In [59]:
sapply(UScrime[c("U1", "U2")], median)

U1 U2 
92 34

In [61]:
with(UScrime, wilcox.test(U1, U2, paired = TRUE))

Warning message in wilcox.test.default(U1, U2, paired = TRUE):
“cannot compute exact p-value with ties”


	Wilcoxon signed rank test with continuity correction

data:  U1 and U2
V = 1128, p-value = 2.464e-09
alternative hypothesis: true location shift is not equal to 0


### Compare more than two group
 - Kruskal-Wallis test (independent in each group)
 - Friedman test (dependent in each group)

In [64]:
states = as.data.frame(cbind(state.region, state.x77))
kruskal.test(Illiteracy ~ state.region, data = states)


	Kruskal-Wallis rank sum test

data:  Illiteracy by state.region
Kruskal-Wallis chi-squared = 22.672, df = 3, p-value = 4.726e-05


### Mann-Whitney U test
Using Mann-Whitney U test compare two group in package npmc

Attention: r-package npmc is not valid now

In [65]:
class = state.region
var = state.x77[, c("Illiteracy")]
mydata = as.data.frame(cbind(class, var))
rm(class, var)
library(npmc)
summary(npmc(mydata), type="BF")
aggregate(mydata, by=list(mydata$class), median)

ERROR: Error in library(npmc): there is no package called ‘npmc’
